In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np

In [2]:
from torch.nn.functional import one_hot

In [208]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.raw_data = dataframe
        self.l = len(self.raw_data)
        self.raw_data['Quality'] = self.raw_data['Quality'].replace(['good', 'bad'], [0.0, 1.0])
        
        self.y_data = self.raw_data['Quality'].values
        self.y_data = torch.from_numpy(self.y_data).type(torch.float32)
#         self.y_data = one_hot(self.y_data)
#         self.y_data = self.y_data.float()
        
        self.raw_data = self.raw_data.drop(labels='Quality', axis = 1)
        self.raw_data = self.raw_data.drop(labels='A_id', axis = 1)
        
        self.x_data = self.raw_data.values
        self.x_data = torch.from_numpy(self.x_data).to(torch.float32)
        
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [209]:
csv_dir = '/home/jetson/pytorch/apple_quality.csv'

csv_data = pd.read_csv(csv_dir)

In [210]:
dataset = CustomDataset(csv_data)

train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

In [211]:
csv_data

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,0.0
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,0.0
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,1.0
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,0.0
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,0.0
...,...,...,...,...,...,...,...,...,...
3995,3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,1.0
3996,3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,0.0
3997,3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,1.0
3998,3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,0.0


In [212]:
train_data, test_data = random_split(dataset, [train_size, test_size])

In [213]:
batch_size = 32

In [214]:
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

In [215]:
for batch, (X, y) in enumerate(test_dataloader):
    print(batch)
    print(f"Shape of X [N, C, H, W]: {X.size()}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

0
Shape of X [N, C, H, W]: torch.Size([32, 7])
Shape of y: torch.Size([32]) torch.float32


In [202]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 28),
            nn.ReLU(),
            nn.Linear(28, 28),
            nn.ReLU(),
            nn.Linear(28,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [203]:
model = NeuralNetwork().to(device)
loss_fn = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [204]:
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=28, bias=True)
    (3): ReLU()
    (4): Linear(in_features=28, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [225]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        pred_p = model(x)
        pred = torch.softmax(pred_p, dim = 1).argmax(1)
        loss = loss_fn(pred_p.squeeze(), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss, current = loss.item(), (batch + 1) *len(x)
    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [234]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #-----> no_grad() gradient를 트래킹 하지않음. 메모리 사용량을 줄여 연산속도를 높이기 위해 사용됨.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred_p = model(X)
            #pred = torch.softmax(pred_p, dim = 1).argmax(1)
            
            test_loss += loss_fn(pred_p.squeeze(), y)
            correct += torch.eq(pred_p.squeeze().round(), y).sum().item()
            break
    test_loss /= num_batches
    print(f'final test_loss : {test_loss}')
    correct /= size
    print(f'final correct : {correct}')
    
    print(f"Test Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [235]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 0.021009  [ 3200/ 3200]
final test_loss : 0.004542195238173008
final correct : 0.0375
Test Error: 
 Accuracy : 3.8%, Avg loss: 0.004542

Epoch 2
---------------------------
loss: 0.015722  [ 3200/ 3200]
final test_loss : 0.005767477676272392
final correct : 0.0375
Test Error: 
 Accuracy : 3.8%, Avg loss: 0.005767

Epoch 3
---------------------------
loss: 0.014333  [ 3200/ 3200]
final test_loss : 0.003461097599938512
final correct : 0.0375
Test Error: 
 Accuracy : 3.8%, Avg loss: 0.003461

Epoch 4
---------------------------
loss: 0.025707  [ 3200/ 3200]
final test_loss : 0.004404718987643719
final correct : 0.0375
Test Error: 
 Accuracy : 3.8%, Avg loss: 0.004405

Epoch 5
---------------------------
loss: 0.024520  [ 3200/ 3200]
final test_loss : 0.005514542106539011
final correct : 0.0375
Test Error: 
 Accuracy : 3.8%, Avg loss: 0.005515

Epoch 6
---------------------------
loss: 0.014345  [ 3200/ 3200]
final test_loss : 0.004727957304567099


KeyboardInterrupt: 

In [16]:
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad(): #-----> no_grad() gradient를 트래킹 하지않음. 메모리 사용량을 줄여 연산속도를 높이기 위해 사용됨.
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y)
        correct += (pred == y).sum().item()
print(f'final test_loss : {test_loss}')
test_loss /= num_batches
print(f'final correct : {correct}')
correct /= size

/home/jetson/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.0086, 0.9914],
        [0.7225, 0.2775],
        [0.2734, 0.7266],
        [0.1877, 0.8123],
        [0.1875, 0.8125],
        [0.8974, 0.1026],
        [0.7628, 0.2372],
        [0.3363, 0.6637],
        [0.4180, 0.5820],
        [0.8394, 0.1606],
        [0.6863, 0.3137],
        [0.3443, 0.6557],
        [0.3230, 0.6770],
        [0.5072, 0.4928],
        [0.0121, 0.9879],
        [0.0313, 0.9687],
        [0.0502, 0.9498],
        [0.3721, 0.6279],
        [0.2533, 0.7467],
        [0.9333, 0.0667],
        [0.7091, 0.2909],
        [0.5899, 0.4101],
        [0.1148, 0.8852],
        [0.0718, 0.9282],
        [0.0832, 0.9168],
        [0.6132, 0.3868],
        [0.0095, 0.9905],
        [0.4991, 0.5009],
        [0.3479, 0.6521],
        [0.7223, 0.2777],
        [0.5617, 0.4383],
        [0.3279, 0.6721]], device='cuda:0') tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
      

RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 1

In [20]:
y.argmax(1)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 1], device='cuda:0')